---

In [1]:
import pandas as pd
import numpy as np
import tweepy
import requests as rq
import json
from yelp.client import Client
from tweepy.parsers import JSONParser
#pip install geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim()

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [6]:
Consumer_Key = "Lxyo498xcrJ87he3pDzEujvaH"
Consumer_Secret = "eTkZJkCsw0u7iYLyNcrg0NN5hAu48r72GzI9u0H009bt2TsujP"
Access_Token = "942094170847526912-kAVi1LeM2T6pqwFNbkCERuDxmgJuH3M"
Access_Token_Secret = "4skwxhl5QUwDsgJDVSm4ynuf6DM8HLQmhhQtVXUP1PWWG"

auth = tweepy.OAuthHandler(Consumer_Key, Consumer_Secret)
auth.set_access_token(Access_Token, Access_Token_Secret)
api = tweepy.API(auth, parser=JSONParser())

In [9]:
target_cities = ["Los Angeles", 
                 "San Francisco",
                 "New York City",
                 "San Diego",
                 "Orlando"]

coordinates = []
for city in target_cities:
    location = geolocator.geocode(city)
    coordinates.append({"Latitude": location.latitude, 
                        "Longitude": location.longitude,
                        "City": city})
    geo_data = []  
    for line in coordinates:
        geo_data.append({"Latitude":line['Latitude'], 
                         "Longitude":line['Longitude']})

        for data in geo_data:
            geo_lat = data['Latitude']
            geo_lng = data['Longitude']

        sentiments_df = pd.read_csv("LA-restaurant.csv")


        compound_list = []
        positive_list = []
        negative_list = []
        neutral_list = []

        for index, row in sentiments_df.iterrows():
            public_tweets = api.search(row["User Name"],
                                       rpp = 100, 
                                       result_type = "recent",
                                       geocode = str(geo_lat)+
                                                 ","+
                                                 str(geo_lng)+
                                                 ",20mi")

            for tweet in public_tweets["statuses"]:
                compound = analyzer.polarity_scores(tweet["text"])["compound"]
                pos = analyzer.polarity_scores(tweet["text"])["pos"]
                neu = analyzer.polarity_scores(tweet["text"])["neu"]
                neg = analyzer.polarity_scores(tweet["text"])["neg"]

                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)

                sentiments_df.set_value(index, city, np.mean(compound_list))

                                     

RateLimitError: [{'message': 'Rate limit exceeded', 'code': 88}]

In [ ]:
new_df

---